## CHEME 5660: Does Markowitz portfolio allocation perform better than alternative approaches?

### Introduction
Fill me in

### Lab 7 setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-7-Markowitz-Portfolio-Allocation`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-7-Markowitz-Portfolio-Allocation/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-7-Markowitz-Portfolio-Allocation/Manifest.toml`


In [2]:
# load req packages -
using Convex
using SCS
using DataFrames
using Dates
using Colors
using Plots
using StatsPlots
using Statistics
using MathOptInterface
using FileIO
using JLD2
using PrettyTables
using Distributions
using LinearAlgebra

# setup paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT, "data");

In [3]:
include("CHEME-5660-Lab-7-CodeLib.jl");

In [4]:
# daily risk-free rate -
r̄ = 0.0403;
risk_free_daily = ((1+r̄)^(1/365) - 1);

# how many days of historical data are we using?
m̂ = 299;
λ̂ = 0.001;

#### Load the OHLC data set
A data hygine issue: we exclude tickers from the data that don't have the same length as the market ticker `SPY`.

In [5]:
# load the JLD2 portfolio data file -
tmp = load(joinpath(_PATH_TO_DATA, "CHEME-5660-Portfolio-10-30-22.jld2"))["dd"];

In [6]:
price_data_dictionary = clean(tmp);

Length violation: META was removed; dim(SPY) = 597 days and dim(META) = 247 days
Length violation: BIIB was removed; dim(SPY) = 597 days and dim(BIIB) = 596 days


In [7]:
# we have these ticker symbols in our data set -
ticker_symbol_array = sort(keys(price_data_dictionary) |> collect);

# how many ticker symbols do we have?
Nₐ = length(ticker_symbol_array);

In [8]:
# Split the data into a training and prediction set
(price_training_dict, price_prediction_dict) = partition(price_data_dictionary, (m̂+1));

In [9]:
# build the set of single index models using the *training* set -
sim_model_dictionary = build(price_training_dict, ticker_symbol_array; 
    m̂ = m̂, λ̂ = λ̂, rf = risk_free_daily);

In [10]:
# compute the excess nreturn for SPY (which is in the data set)
(Rₘ, R̂ₘ, W, μᵦ, pᵦ) = compute_excess_return(price_training_dict["SPY"]; 
    m = m̂, rf = risk_free_daily, λ = λ̂);

In [11]:
# compute the μ_vector -
μ_vector = μ(sim_model_dictionary, R̂ₘ, ticker_symbol_array);

In [12]:
# compute Σ -
Σ_array = Σ(sim_model_dictionary, R̂ₘ, ticker_symbol_array);

In [13]:
# define the returns that we are going to simulate -
target_return_array = 100 .*range(0.0, step=0.0001, stop = 0.003) |> collect # percentage
L = length(target_return_array);

In [ ]:
# Initialize MinVar array
# row: return
# col 1 -> risk
# col 2 -> return
# col 3 ... N -> allocation
MinVarArray = Array{Float64,2}(undef,L, (Nₐ + 2));

# main loop: pick a return, run the calculation. If solver converged, store results
for i = 1:L

    tr = target_return_array[i]
    (status_flag, ω, opt_val, ret_val) = compute_minvar_portfolio_allocation(μ_vector, Σ_array, tr; w_lower = 0.0);

    if (status_flag == MathOptInterface.OPTIMAL)
        
        MinVarArray[i,1] = sqrt(opt_val);
        MinVarArray[i,2] = ret_val;
        
        # capture the allocation
        for a ∈ 1:Nₐ
            MinVarArray[i,2+a] = ω[a]
        end
    end
end

In [ ]:
# Initialize MinVarRiskFree array
# row: return
# col 1 -> risk
# col 2 -> return
# col 3 ... N -> allocation
MinVarRiskFreeArray = Array{Float64,2}(undef, L, (Nₐ + 2));

# main loop: pick a return, run the calculation. If solver converged, store results
for i = 1:L

    tr = target_return_array[i]
    (status_flag, ω, opt_val, ret_val) = compute_minvar_portfolio_allocation_risk_free(μ_vector, Σ_array, tr;
        risk_free_return = risk_free_daily, w_lower =  0.0);

    if (status_flag == MathOptInterface.OPTIMAL)
        
        # capture the risk and reward values -
        MinVarRiskFreeArray[i,1] = sqrt(opt_val);
        MinVarRiskFreeArray[i,2] = ret_val;
        
        # capture the allocation
        for a ∈ 1:Nₐ
            MinVarRiskFreeArray[i,2+a] = ω[a]
        end
    end
end

In [ ]:
# plot the risk-only portfolio -
plot(MinVarArray[:,1], MinVarArray[:,2], label="", c=colorant"#EF4035", legend=:bottomright, bg="floralwhite", 
    background_color_outside="white", framestyle = :box, fg_legend = :transparent, lw=3, minorticks=0.05, xlim=(0.0,3.0))
scatter!(MinVarArray[:,1], MinVarArray[:,2], label="Possible portfolio", mc=:white, msc=colorant"#EF4035")

# plot the CAL -
plot!(MinVarRiskFreeArray[:,1],  MinVarRiskFreeArray[:,2], lw=3, c=colorant"#89CCE2", label="Capitol Allocation Line (CAL)")

# axis labels -
xlabel!("Risk measure (σ-risk)", fontsize=18)
ylabel!("Exepcted excess daily return (%)", fontsize=18)

In [ ]:
# make a allocation table for risky+risk free asset case -

# what portfolio index do we need?
portfolio_index_rf = findall(x->x<=0.60, MinVarRiskFreeArray[:,1])[end]
δ = 0.001; # what is my cutoff?

# find the indexes of the assets that are "not small" -
idx_not_small_rf = findall(x-> abs(x) >= δ, MinVarRiskFreeArray[portfolio_index_rf,3:end])
A = length(idx_not_small_rf);

# setup table -
allocation_table_data_rf = Array{Any,2}(undef, A+1, 4);
for a ∈ 1:A
    
    # grab the data -
    idx = idx_not_small_rf[a];
    ticker = ticker_symbol_array[idx]
    ωₐ = MinVarRiskFreeArray[portfolio_index_rf,(idx .+ 2)];

    # package -
    allocation_table_data_rf[a,1] = ticker;
    allocation_table_data_rf[a,2] = ωₐ
    allocation_table_data_rf[a,3] = μ_vector[idx];
    allocation_table_data_rf[a,4] = Σ_array[idx,idx];
end

# add a total row -
allocation_table_data_rf[end,1] = "Total"
allocation_table_data_rf[end,2] = sum(MinVarRiskFreeArray[portfolio_index_rf, (idx_not_small_rf .+ 2)])
allocation_table_data_rf[end,3] = MinVarRiskFreeArray[portfolio_index_rf,2];
allocation_table_data_rf[end,4] = MinVarRiskFreeArray[portfolio_index_rf,1];

# header -
allocation_table_header_rf = (["Ticker", "weight", "E(r)", "σ"])

# show the table -
pretty_table(allocation_table_data_rf; header = allocation_table_header_rf)

In [ ]:
RR = compute_realized_excess_return(price_prediction_dict, ticker_symbol_array; rf = risk_free_daily);

In [ ]:
RRT = transpose(RR)

In [ ]:
ω =  MinVarRiskFreeArray[portfolio_index_rf,3:end]
Z = RRT*ω

In [ ]:
Wₒ = 10000.0;
NZ = length(Z);
WM = Array{Float64,2}(undef, NZ, 4);
R_SPY = compute_realized_excess_return(price_prediction_dict, ["SPY"]; rf = risk_free_daily)
R_AMD = compute_realized_excess_return(price_prediction_dict, ["MMM"]; rf = risk_free_daily)
WM[1,1] = 0;
WM[1,2] = Wₒ
WM[1,3] = Wₒ
WM[1,4] = Wₒ
for i ∈ 1:NZ-1
    
    WM[i+1,1] = i;
    WM[i+1,2] = (1+(Z[i] + risk_free_daily))*WM[i,2]
    WM[i+1,3] = (1+(R_SPY[i] + risk_free_daily))*WM[i,3];
    WM[i+1,4] = (1+(R_AMD[i] + risk_free_daily))*WM[i,4];
end

In [ ]:
plot(WM[:,1], WM[:,2]);
plot!(WM[:,1], WM[:,3], c=:red)
plot!(WM[:,1], WM[:,4], c=:green)